In [3]:
import re
import os
from pathlib import Path

In [17]:
mat_ext_sig_re = re.compile(
    r'''^((?:[^/]*/)*)?        # Path
        ((?:[@+]\w+\.)*)       # Namespace
        (\w+)                  # Filename
        (?:\((.*)\))?$         # Arguments
        ''', re.VERBOSE)

mat_re_namespace = re.compile(
    r'([@+]\w+)\.', re.VERBOSE  # Namespace list
)

mat_prop_re = re.compile(
    r'''^
    (\w+)                       # argument
    (?:\s*\(([\s\d,\:]+)\))?      # size
    (?:\s*(\w+))?               # argument class
    (?:\s*\{([\s\w,]+)\})?      # validators
    (?:\s*\=\s*(\w+))?          # default value
    ''', re.VERBOSE
)

def parse_autodoc_input(prompt:str, matlab_root_dir:str, src_path:str):

    (path, namespacePath, filename, arguments) = mat_ext_sig_re.match(prompt).groups()
    namespace = mat_re_namespace.findall(namespacePath)

    basepath = Path(matlab_root_dir if matlab_root_dir else src_path)
    abspath = (basepath / Path(path or '.')).resolve()
    filepath = (abspath / '/'.join(namespace)).resolve()

    return (filepath, filename, arguments, abspath, namespace)

def parse_property(prop_str:str):
    (arg, size, arg_class, validators, default) = mat_prop_re.match(prop_str).groups()
    if size:
        size = [dim.strip() for dim in size.split(',')]
    if validators:
        validators = [validator.strip() for validator in validators.split(',')]
    return (arg, size, arg_class, validators, default)


In [24]:
parse_property('arg2 (1,:) logical {MustBeReal} = True')

('arg2', ['1', ':'], 'logical', ['MustBeReal'], 'True')

In [25]:
arg = '../@package.+main.test(arg1, arg2)'
matlab_root_dir = 'C:/Users'
src_path = os.getcwd()

parse_autodoc_input(arg, matlab_root_dir, src_path)

(WindowsPath('C:/@package/+main'),
 'test',
 'arg1, arg2',
 WindowsPath('C:/'),
 ['@package', '+main'])